# Introduction
<hr style = "border:2px solid black" ></hr>


**What?** Intoduction on Pandera



# Import modules
<hr style = "border:2px solid black" ></hr>

In [1]:
# pip install pandera 
# pip inarLL ipytest
import pandas as pd
import pandera as pa
from pandera import Column, Check
from pandera.typing import Series
import ipytest
import numpy as np

# What is Pandera
<hr style = "border:2px solid black" ></hr>


- In a data science project, it is not only important to test your functions, but it is also important to test your data to make sure they work as you expected.
- One option is Great Expectations but for a small data science project, this can be an overkill.
- An alternative is a simple Python library for validating a pandas DataFrame called: `pandera`.



# Dummy dataset creation
<hr style = "border:2px solid black" ></hr>

In [2]:
fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": [2, 1, 3, 4],
    }
)

fruits

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4


# Simple checks
<hr style = "border:2px solid black" ></hr>


- Scenario: your manager told you that there can only be certain fruits and stores in the dataset, and the price must be less than 4.
- To make sure your data follow these conditions, checking your data manually can cost too much time, especially when your data is big. Is there a way that you can automate this process?
- That is when Pandera comes in handy. Specifically, we:
    - Create multiple tests for the entire dataset using `DataFrameSchema`
    - Create multiple tests for each column using `Column`
    - Specify the type of test using `Check`
- Since not all values in the column price are less than 4, the test fails.



In [3]:
available_fruits = ["apple", "banana", "orange"]
nearby_stores = ["Aldi", "Walmart"]

In [4]:

schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(int, Check.less_than(4)),
    }
)
schema.validate(fruits)

SchemaError: <Schema Column(name=price, type=DataType(int64))> failed element-wise validator 0:
<Check less_than: less_than(4)>
failure cases:
   index  failure_case
0      3             4


- If we use `5` instead of `4` then the test will pass.



In [5]:
schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(int, Check.less_than(5)),
    }
)
schema.validate(fruits)

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4



- We can also create custom checks using lambda. 
- In the code below, we are going to check if the sum of the column price is less than 20.



In [6]:
schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(
            int, [Check.less_than(5), Check(lambda price: sum(price) < 20)]
        ),
    }
)
schema.validate(fruits)

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4


# Schema Model


- When our tests are complicated, using dataclass can make our tests look much cleaner than using a dictionary. 
- Luckily, Pandera also allows us to create tests using a dataclass instead of a dictionary.



In [7]:
class Schema(pa.SchemaModel):
    name: Series[str] = pa.Field(isin=available_fruits)
    store: Series[str] = pa.Field(isin=nearby_stores)
    price: Series[int] = pa.Field(le=5)

    @pa.check("price")
    def price_sum_lt_20(cls, price: Series[int]) -> Series[bool]:
        return sum(price) < 20


Schema.validate(fruits)

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,Aldi,4


# Validation Decorator 

## Check Input

In [8]:


ipytest.autoconfig()

In [9]:
fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": [2, 1, 3, 4],
    }
)

schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(int, Check.less_than(5)),
    }
)


def get_total_price(fruits: pd.DataFrame, schema: pa.DataFrameSchema):
    validated = schema.validate(fruits)
    return validated["price"].sum()


get_total_price(fruits, schema)

10

In [10]:
%%ipytest -qq
def test_get_total_price():
    fruits = pd.DataFrame({'name': ['apple', 'banana'], 'store': ['Aldi', 'Walmart'], 'price': [1, 2]})
    
    schema = pa.DataFrameSchema(
        {
            "name": Column(str, Check.isin(available_fruits)),
            "store": Column(str, Check.isin(nearby_stores)),
            "price": Column(int, Check.less_than(5)),
        }
    )
    assert get_total_price(fruits, schema) == 3

.                                                                                            [100%]


In [ ]:
from pandera import check_input, check_output, check_io

fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": ["2", "1", "3", "4"],
    }
)


@check_input(schema)
def get_total_price(fruits: pd.DataFrame):
    return fruits.price.sum()


get_total_price(fruits)

In [11]:
%%ipytest -qq
def test_get_total_price():
    fruits = pd.DataFrame({'name': ['apple', 'banana'], 'store': ['Aldi', 'Walmart'], 'price': [1, 2]})
    assert get_total_price(fruits) == 3

F                                                                                            [100%]
============================================= FAILURES =============================================
_______________________________________ test_get_total_price _______________________________________

    def test_get_total_price():
        fruits = pd.DataFrame({'name': ['apple', 'banana'], 'store': ['Aldi', 'Walmart'], 'price': [1, 2]})
>       assert get_total_price(fruits) == 3
E       TypeError: get_total_price() missing 1 required positional argument: 'schema'

/var/folders/5x/lyqr8wv507n43bnwpsz4dq_c0000gn/T/ipykernel_87717/2601245265.py:3: TypeError
===================================== short test summary info ======================================
FAILED tmpan9spb4c.py::test_get_total_price - TypeError: get_total_price() missing 1 required pos...


## Check Output

In [12]:
fruits_nearby = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": [2, 1, 3, 4],
    }
)

fruits_faraway = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Whole Foods", "Whole Foods", "Schnucks", "Schnucks"],
        "price": [3, 2, 4, 5],
    }
)

out_schema = pa.DataFrameSchema(
    {"store": Column(str, Check.isin(["Aldi", "Walmart", "Whole Foods", "Schnucks"]))}
)


@check_output(out_schema)
def combine_fruits(fruits_nearby: pd.DataFrame, fruits_faraway: pd.DataFrame):
    fruits = pd.concat([fruits_nearby, fruits_faraway])
    return fruits


combine_fruits(fruits_nearby, fruits_faraway)

NameError: name 'check_output' is not defined

## Check Both

In [13]:
in_schema = pa.DataFrameSchema({"store": Column(str)})

out_schema = pa.DataFrameSchema(
    {"store": Column(str, Check.isin(["Aldi", "Walmart", "Whole Foods", "Schnucks"]))}
)


@check_io(fruits_nearby=in_schema, fruits_faraway=in_schema, out=out_schema)
def combine_fruits(fruits_nearby: pd.DataFrame, fruits_faraway: pd.DataFrame):
    fruits = pd.concat([fruits_nearby, fruits_faraway])
    return fruits


combine_fruits(fruits_nearby, fruits_faraway)

NameError: name 'check_io' is not defined

# Other Arguments for Column Validation

## Deal with Null Values

In [14]:
fruits = fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", np.nan],
        "price": [2, 1, 3, 4],
    }
)

fruits

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,NaN,4


In [15]:
schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(str, Check.isin(nearby_stores)),
        "price": Column(int, Check.less_than(5)),
    }
)
schema.validate(fruits)

SchemaError: non-nullable series 'store' contains null values:
3    NaN
Name: store, dtype: object

In [16]:
schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(str, Check.isin(nearby_stores), nullable=True),
        "price": Column(int, Check.less_than(5)),
    }
)
schema.validate(fruits)

,name,store,price
0,apple,Aldi,2
1,banana,Walmart,1
2,apple,Walmart,3
3,orange,NaN,4


## Deal with Duplicates

In [17]:
schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store": Column(
            str, Check.isin(nearby_stores), nullable=True, allow_duplicates=False
        ),
        "price": Column(int, Check.less_than(5)),
    }
)
schema.validate(fruits)

TypeError: __init__() got an unexpected keyword argument 'allow_duplicates'

## Convert Data Types

In [18]:
fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "price": [2, 1, 3, 4],
    }
)

schema = pa.DataFrameSchema({"price": Column(str, coerce=True)})
validated = schema.validate(fruits)
validated.dtypes

name     object
store    object
price    object
dtype: object

## Patern Matching

In [19]:
favorite_stores = ["Aldi", "Walmart", "Whole Foods", "Schnucks"]

fruits = pd.DataFrame(
    {
        "name": ["apple", "banana", "apple", "orange"],
        "store_nearby": ["Aldi", "Walmart", "Walmart", "Aldi"],
        "store_far": ["Whole Foods", "Schnucks", "Whole Foods", "Schnucks"],
    }
)

schema = pa.DataFrameSchema(
    {
        "name": Column(str, Check.isin(available_fruits)),
        "store_+": Column(str, Check.isin(favorite_stores), regex=True),
    }
)
schema.validate(fruits)

,name,store_nearby,store_far
0,apple,Aldi,Whole Foods
1,banana,Walmart,Schnucks
2,apple,Walmart,Whole Foods
3,orange,Aldi,Schnucks


# Export and Load From a YAML file


- Using a YAML file is a neat way to show your tests to colleagues who don’t know Python. 
- We can keep a record of all validations in a YAML file using 



## Export

In [20]:
yaml_schema = schema.to_yaml()
print(yaml_schema)

schema_type: dataframe
version: 0.10.1
columns:
  name:
    dtype: str
    nullable: false
    checks:
      isin:
      - apple
      - banana
      - orange
    unique: false
    coerce: false
    required: true
    regex: false
  store_+:
    dtype: str
    nullable: false
    checks:
      isin:
      - Aldi
      - Walmart
      - Whole Foods
      - Schnucks
    unique: false
    coerce: false
    required: true
    regex: true
checks: null
index: null
coerce: false
strict: false
unique: null



In [23]:
from pathlib import Path

f = Path("schema.yml")
f.touch()
f.write_text(yaml_schema)

504

## Load

In [24]:
with f.open() as file:
    yaml_schema = file.read()

In [25]:
schema = pa.io.from_yaml(yaml_schema)
schema

<Schema DataFrameSchema(columns={'name': <Schema Column(name=name, type=DataType(str))>, 'store_+': <Schema Column(name=store_+, type=DataType(str))>}, checks=[], index=None, coerce=False, dtype=None, strict=False, name=None, ordered=False, unique_column_names=False)>

# References
<hr style = "border:2px solid black" ></hr>


- [Blog article](https://towardsdatascience.com/validate-your-pandas-dataframe-with-pandera-2995910e564)
- [GitHub code](https://github.com/khuyentran1401/Data-science/tree/master/data_science_tools/pandera_example)

